In [33]:
from datasets import load_dataset  # HuggingFace datasets

from gpt2 import tokenizer
from data_utils import tokenize_ds, DataLoader

## Load Dataset

In [34]:
context_length, batch_size = 1024, 16

ds = load_dataset("NeelNanda/pile-10k", split="train").remove_columns("meta")
ds_tokens = tokenize_ds(ds, tokenizer, max_length=context_length, num_proc=None)
ds_split = ds_tokens.train_test_split(test_size=1000)

ldr_train = DataLoader(ds_split["train"], batch_size, True)
ldr_test = DataLoader(ds_split["test"], batch_size)

print("Text:\t", ds[0]["text"][:40] + "...")
print("Tokens:\t", ",".join([str(i) for i in ds_tokens[0]["tokens"][:10]]) + "...")


Text:	 It is done, and submitted. You can play ...
Tokens:	 50256,1026,318,1760,11,290,8948,13,921,460...
